# ***Item-Item***

In [39]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.metrics.pairwise import cosine_similarity

In [40]:
# def preprocess_ratings(ratings):
#     # Check for missing value
#     if ratings[['customer_id', 'product_id', 'stars']].isna().any().any():
#         ratings = ratings.dropna(subset=['customer_id', 'product_id', 'stars'])

#     # Validate ratings (ensure stars are between 0 and 5)
#     invalid_ratings = ratings[(ratings['stars'] < 0) | (ratings['stars'] > 5)]
#     if not invalid_ratings.empty:
#         ratings = ratings[(ratings['stars'] >= 0) & (ratings['stars'] <= 5)]

#     # Handle duplicates by averaging ratings for the same customer_id and product_id
#     # ratings = ratings.groupby(['customer_id', 'product_id']).agg({'stars': 'max'}).reset_index()
#     # ratings = ratings.groupby(['customer_id', 'product_id']).agg({'stars': 'mean'}).reset_index()

#     # Ensure data types are correct
#     ratings['customer_id'] = ratings['customer_id'].astype(int)
#     ratings['product_id'] = ratings['product_id'].astype(int)
#     ratings['stars'] = ratings['stars'].astype(float)

#     return ratings

In [41]:
import pandas as pd

def preprocess_ratings(ratings: pd.DataFrame) -> pd.DataFrame:
    # 1. Tạo một bản sao để tránh thay đổi DataFrame gốc
    processed_ratings = ratings.copy()

    # 2. Kiểm tra sự tồn tại của các cột cần thiết
    required_cols = ['customer_id', 'product_id', 'stars', 'time']
    if not all(col in processed_ratings.columns for col in required_cols):
        raise ValueError(f"DataFrame đầu vào phải chứa các cột: {required_cols}")

    # 3. Xử lý giá trị thiếu
    initial_count = len(processed_ratings)
    processed_ratings.dropna(subset=required_cols, inplace=True)
    if len(processed_ratings) < initial_count:
        print(f"Đã loại bỏ {initial_count - len(processed_ratings)} dòng có giá trị thiếu.")

    # 4. Xác thực giá trị rating (ví dụ: trong khoảng 0-5)
    initial_count = len(processed_ratings)
    processed_ratings = processed_ratings[(processed_ratings['stars'] >= 0) & (processed_ratings['stars'] <= 5)]
    if len(processed_ratings) < initial_count:
        print(f"Đã loại bỏ {initial_count - len(processed_ratings)} dòng có rating không hợp lệ.")

    # 5. Xử lý các đánh giá trùng lặp bằng cách giữ lại đánh giá mới nhất
    print(f"Số lượng đánh giá trước khi xử lý trùng lặp: {len(processed_ratings)}")

    # Chuyển đổi cột 'time' sang kiểu datetime để đảm bảo sắp xếp đúng
    processed_ratings['time'] = pd.to_datetime(processed_ratings['time'])

    # Sắp xếp toàn bộ DataFrame theo thời gian, từ cũ nhất đến mới nhất
    processed_ratings.sort_values(by='time', ascending=True, inplace=True)

    # Loại bỏ các bản ghi trùng lặp trên cặp (customer_id, product_id),
    # và chỉ giữ lại bản ghi cuối cùng ('last') trong mỗi nhóm.
    # Vì đã sắp xếp theo thời gian, 'last' chính là bản ghi mới nhất.
    processed_ratings.drop_duplicates(subset=['customer_id', 'product_id'], keep='last', inplace=True)

    # 6. Đảm bảo kiểu dữ liệu cuối cùng và loại bỏ cột không cần thiết
    processed_ratings['customer_id'] = processed_ratings['customer_id'].astype(int)
    processed_ratings['product_id'] = processed_ratings['product_id'].astype(int)
    processed_ratings['stars'] = processed_ratings['stars'].astype(float)

    processed_ratings.drop(columns=['time'], inplace=True)

    return processed_ratings

In [42]:
def validate_products(products: pd.DataFrame, ratings: pd.DataFrame) -> pd.DataFrame:
    """
    Validate the products DataFrame and ensure consistency with ratings.
    """
    # Check for missing values
    if products[['id', 'name']].isna().any().any():
        products['name'] = products['name'].fillna('Unknown')

    # Ensure unique product IDs
    if products['id'].duplicated().any():
        products = products.drop_duplicates(subset='id', keep='first')

    # Check if all product_ids in ratings exist in products
    missing_products = set(ratings['product_id']) - set(products['id'])
    if missing_products:
        # Optionally, filter out ratings with missing products
        ratings = ratings[ratings['product_id'].isin(products['id'])]

    # Ensure data types
    products['id'] = products['id'].astype(int)
    products['name'] = products['name'].astype(str)

    return products, ratings

In [43]:
def create_user_item_matrix(ratings: pd.DataFrame) -> pd.DataFrame:
    """
    Create a user-item matrix from ratings.

    Args:
        ratings (pd.DataFrame): Cleaned ratings DataFrame

    Returns:
        pd.DataFrame: User-item matrix (rows: customer_id, columns: product_id, values: stars)
    """
    user_item_matrix = ratings.pivot(index='customer_id', columns='product_id', values='stars')
    return user_item_matrix

In [44]:
def mean_center_matrix(user_item_matrix: pd.DataFrame) -> pd.DataFrame:
    """
    Mean-center the user-item matrix by subtracting the mean rating for each user.
    """
    # Calculate mean rating per user (ignoring NaN)
    user_means = user_item_matrix.mean(axis=1, skipna=True)

    # Subtract mean from each user's ratings
    user_item_matrix_centered = user_item_matrix.sub(user_means, axis=0)

    # Replace NaN with 0 (as per the collaborative filtering document)
    user_item_matrix_centered = user_item_matrix_centered.fillna(0)

    return user_item_matrix_centered, user_means

In [45]:
products = pd.read_csv("products_info.csv")
ratings = pd.read_csv("ratings_with_time.csv")

ratings = preprocess_ratings(ratings)
cleaned_products, cleaned_ratings = validate_products(products, ratings)

train_df, test_df = train_test_split(cleaned_ratings, test_size=0.3, random_state=42)
print(f"Training set: {len(train_df)}")
print(f"Test set: {len(test_df)}\n")

Số lượng đánh giá trước khi xử lý trùng lặp: 1500
Training set: 900
Test set: 387



In [46]:
# Create user-item matrix
user_item_matrix = create_user_item_matrix(cleaned_ratings)

# Mean-center the user-item matrix
user_item_matrix_centered, user_means = mean_center_matrix(user_item_matrix)

# Compute item-item similarity (using Pearson correlation on mean-centered matrix)
item_similarity_df = user_item_matrix_centered.corr(method='pearson')
item_similarity_df = item_similarity_df.fillna(0)

In [47]:
user_item_matrix.to_csv("user_item_matrix.csv")
user_item_matrix_centered.to_csv("user_item_matrix_centered.csv")
user_means.to_csv(f"user_means.csv")
item_similarity_df.to_csv("item_similarity.csv")

In [48]:
def evaluate_model(predictions_df):
    """
    Đánh giá mô hình bằng cách tính toán RMSE.

    Args:
        predictions_df (pd.DataFrame): DataFrame chứa cột 'stars' (thực tế)
                                      và 'predicted_rating' (dự đoán).

    Returns:
        float: Giá trị RMSE.
    """
    if predictions_df.empty:
        print("Không có dự đoán nào để đánh giá.")
        return float('inf')

    # Loại bỏ các dòng không thể dự đoán được (predicted_rating là NaN)
    predictions_df.dropna(subset=['predicted_rating'], inplace=True)

    if predictions_df.empty:
        print("Tất cả các dự đoán đều là NaN.")
        return float('inf')

    # Tính Mean Squared Error (MSE)
    mse = mean_squared_error(predictions_df['stars'], predictions_df['predicted_rating'])

    # Tính Root Mean Squared Error (RMSE)
    rmse = np.sqrt(mse)

    return rmse

In [49]:
def generate_predictions_for_test_set(test_df, user_item_matrix_train, item_similarity, user_means_train, k=10):
    predictions = []
    for _, row in test_df.iterrows():
        user_id = int(row['customer_id'])
        item_id = int(row['product_id'])
        predicted_rating = np.nan

        if user_id in user_item_matrix_train.index and item_id in item_similarity.index:
            rated_items = user_item_matrix_train.loc[user_id].dropna()
            user_mean = user_means_train.loc[user_id]

            # Lấy vector tương đồng của item cần dự đoán
            similarities_to_target = item_similarity.get(item_id, pd.Series(dtype='float64'))

            # Chỉ xét các item tương tự MÀ user đã đánh giá
            # và chúng cũng phải có trong ma trận tương đồng
            common_items = rated_items.index.intersection(similarities_to_target.index)

            if not common_items.empty:
                similarities_to_rated_items = similarities_to_target.loc[common_items]
                top_k_similar = similarities_to_rated_items.nlargest(k)

                numerator, denominator = 0, 0
                for sim_item, sim_score in top_k_similar.items():
                    deviation = rated_items.loc[sim_item] - user_mean
                    numerator += sim_score * deviation
                    denominator += abs(sim_score)

                if denominator > 0:
                    predicted_deviation = numerator / denominator
                    predicted_rating = user_mean + predicted_deviation

        predictions.append(predicted_rating)

    test_df['predicted_rating'] = predictions
    return test_df

In [50]:
def calculate_ranking_metrics(predictions_df, k=10, threshold=4.0):
    """
    Tính toán các chỉ số Precision@K và Recall@K trung bình cho tất cả người dùng.

    Args:
        predictions_df (pd.DataFrame): DataFrame chứa các cột 'customer_id', 'product_id',
                                      'stars' (thực tế), và 'predicted_rating'.
        k (int): Số lượng item hàng đầu trong danh sách gợi ý.
        threshold (float): Ngưỡng rating để coi một item là 'liên quan'.

    Returns:
        tuple: Một tuple chứa (precision_at_k_mean, recall_at_k_mean).
    """
    # Tạo một dictionary để lưu trữ gợi ý và các item liên quan cho mỗi user
    user_data = {}

    # Lọc các dự đoán hợp lệ
    valid_predictions = predictions_df.dropna(subset=['predicted_rating'])

    for _, row in valid_predictions.iterrows():
        user_id = int(row['customer_id'])
        product_id = int(row['product_id'])
        rating = row['stars']

        # Khởi tạo user nếu chưa có
        if user_id not in user_data:
            user_data[user_id] = {'recommendations': [], 'relevant_items': []}

        if rating >= threshold:
            user_data[user_id]['relevant_items'].append(product_id)

    valid_predictions = predictions_df.dropna(subset=['predicted_rating'])
    for _, row in valid_predictions.iterrows():
        user_id = int(row['customer_id'])
        product_id = int(row['product_id'])
        predicted_rating = row['predicted_rating']

        user_data[user_id]['recommendations'].append((predicted_rating, product_id))

    # Tính Precision và Recall cho mỗi người dùng
    precisions = []
    recalls = []

    for user_id, data in user_data.items():
        # Bỏ qua nếu user không có item nào liên quan trong test set
        if not data['relevant_items']:
            continue

        # Sắp xếp các gợi ý theo điểm dự đoán giảm dần
        data['recommendations'].sort(key=lambda x: x[0], reverse=True)

        # Lấy top K sản phẩm được gợi ý
        top_k_recs = {item_id for _, item_id in data['recommendations'][:k]}

        # Lấy danh sách các sản phẩm liên quan thực tế
        relevant_items = set(data['relevant_items'])

        # Đếm số sản phẩm gợi ý đúng (hit)
        hits = len(top_k_recs & relevant_items)
        print(f"User {user_id}: Hits={hits}, Relevant={len(relevant_items)}")
        # Tính Precision@K
        precision_at_k = hits / k if k > 0 else 0.0
        precisions.append(precision_at_k)

        # Tính Recall@K
        recall_at_k = hits / len(relevant_items) if relevant_items else 0.0
        recalls.append(recall_at_k)

    # Trả về trung bình cộng của tất cả
    mean_precision = np.mean(precisions) if precisions else 0.0
    mean_recall = np.mean(recalls) if recalls else 0.0

    return mean_precision, mean_recall

In [51]:
def recommend_items_item_item(user_id: int, user_item_matrix: pd.DataFrame, item_similarity: pd.DataFrame,
                              products: pd.DataFrame, user_means: pd.Series, n: int = 10, k: int = 10) -> pd.DataFrame:
    """
    Generate top-n item-item collaborative filtering recommendations for a user,
    with the final result sorted by predicted_rating.

    Args:
        user_id (int): ID of the target user
        user_item_matrix (pd.DataFrame): User-item matrix
        item_similarity (pd.DataFrame): Item-item similarity matrix
        products (pd.DataFrame): Products DataFrame
        user_means (pd.Series): User mean ratings (not used in this specific logic, but kept for signature consistency)
        n (int): Number of recommendations to return
        k (int): Number of similar items to consider

    Returns:
        pd.DataFrame: Recommended products with predicted ratings, sorted by predicted_rating descending.
    """
    # Identify unrated items
    user_ratings = user_item_matrix.loc[user_id]
    unrated_items = user_ratings[user_ratings.isna()].index
    predictions = []

    # Predict rating for each unrated item
    for item in unrated_items:
        rated_items = user_ratings.dropna().index

        # Lấy vector tương đồng và xử lý trường hợp item không có trong ma trận
        item_sim_vector = item_similarity.get(item)
        if item_sim_vector is None:
            continue

        similar_items = item_sim_vector.loc[item_sim_vector.index.intersection(rated_items)].nlargest(k)

        numerator, denominator = 0, 0
        for sim_item, sim_score in similar_items.items():
            if not np.isnan(user_item_matrix.loc[user_id, sim_item]):
                numerator += sim_score * user_item_matrix.loc[user_id, sim_item]
                denominator += abs(sim_score)

        if denominator > 0:
            predicted_rating = (numerator / denominator)
            predictions.append((item, predicted_rating))

    # Sort and select top-n recommendations
    predictions.sort(key=lambda x: x[1], reverse=True)
    top_recommendations = predictions[:n]

    if not top_recommendations:
        return pd.DataFrame()

    # Create a DataFrame from the top recommendations
    recs_df = pd.DataFrame(top_recommendations, columns=['id', 'predicted_rating'])

    # Merge with product information
    final_recs = pd.merge(recs_df, products[['id', 'name']], on='id')

    # Sắp xếp DataFrame theo 'predicted_rating' giảm dần.
    final_recs = final_recs.sort_values(by='predicted_rating', ascending=False)

    return final_recs

In [66]:
#  Huấn luyện Mô hình trên Training Set
# Tạo ma trận tiện ích CHỈ từ dữ liệu training
user_item_matrix_train = create_user_item_matrix(train_df)

# Tính user_means TỪ BỘ TRAINING
user_means_train = user_item_matrix_train.mean(axis=1)

# Chuẩn hóa ma trận training
user_item_matrix_train_centered, _ = mean_center_matrix(user_item_matrix_train)

# Tính ma trận tương đồng sản phẩm CHỈ từ dữ liệu training
print("Huấn luyện mô hình: Tính toán Item-Item Similarity từ Training Set")
item_similarity_df = user_item_matrix_train_centered.corr(method='pearson')
item_similarity_df.fillna(0, inplace=True)

# --- 4. Tạo Dự đoán và Đánh giá trên Test Set ---
print("Tạo dự đoán cho Test Set")
predictions_df = generate_predictions_for_test_set(
    test_df.copy(),
    user_item_matrix_train,
    item_similarity_df,
    user_means_train,
    k=10)

print(predictions_df.dropna(subset=['predicted_rating']).head(10))

Huấn luyện mô hình: Tính toán Item-Item Similarity từ Training Set
Tạo dự đoán cho Test Set
      customer_id  product_id  stars  predicted_rating
1455           15         116    3.2          4.169998
381            31          69    0.7          1.499773
1101            2         158    2.5          4.687061
477            23          97    4.1          4.269978
1413           34          58    4.4          3.767044
1082           36         182    4.7          3.262360
286            18         118    0.3          2.882671
554            46         182    4.2          2.904052
343             7          97    4.4          3.327935
490            45         110    4.9          4.546369


# **Evaluate**

In [54]:
print("\n--- Evaluate model ---")
rmse_score = evaluate_model(predictions_df)
print(f"Root Mean Square Error (RMSE): {rmse_score:.4f}")


--- Evaluate model ---
Root Mean Square Error (RMSE): 1.6930


In [70]:
# Precision@K và Recall@K
k_value = 10
relevance_threshold = 4.0

precision, recall = calculate_ranking_metrics(
  predictions_df,
  k=k_value,
  threshold=relevance_threshold)

User 15: Hits=3, Relevant=3
User 31: Hits=6, Relevant=6
User 2: Hits=3, Relevant=3
User 23: Hits=6, Relevant=6
User 34: Hits=3, Relevant=3
User 36: Hits=8, Relevant=8
User 18: Hits=6, Relevant=6
User 46: Hits=3, Relevant=3
User 7: Hits=3, Relevant=3
User 45: Hits=3, Relevant=3
User 24: Hits=4, Relevant=4
User 9: Hits=3, Relevant=3
User 20: Hits=2, Relevant=2
User 43: Hits=2, Relevant=2
User 26: Hits=5, Relevant=5
User 47: Hits=6, Relevant=7
User 32: Hits=7, Relevant=7
User 14: Hits=6, Relevant=6
User 40: Hits=6, Relevant=6
User 6: Hits=3, Relevant=3
User 35: Hits=5, Relevant=5
User 25: Hits=4, Relevant=4
User 28: Hits=4, Relevant=4
User 38: Hits=7, Relevant=7
User 11: Hits=3, Relevant=3
User 41: Hits=7, Relevant=9
User 33: Hits=6, Relevant=6
User 42: Hits=2, Relevant=2
User 19: Hits=4, Relevant=4
User 50: Hits=5, Relevant=5
User 17: Hits=7, Relevant=7
User 21: Hits=4, Relevant=4
User 30: Hits=2, Relevant=2
User 12: Hits=1, Relevant=1
User 1: Hits=2, Relevant=2
User 39: Hits=6, Relevant

In [68]:
print(f"\n--- All metrics rating (with K={k_value} và and revevant threshold >= {relevance_threshold}) ---")
print(f"Precision at Top-{k_value} (Precision@{k_value}): {precision:.2%}")

print(f"\nRecall at Top-{k_value} (Recall@{k_value}): {recall:.2%}")


--- All metrics rating (with K=10 và and revevant threshold >= 4.0) ---
Precision at Top-10 (Precision@10): 41.60%

Recall at Top-10 (Recall@10): 98.83%


In [53]:
# Example: Generate recommendations for user 1
recommendations = recommend_items_item_item(
    user_id=1,
    user_item_matrix=user_item_matrix,
    item_similarity=item_similarity_df,
    products=cleaned_products,
    user_means=user_means,
    n=10,
    k=10)

recommendations

,id,predicted_rating,name
0,176,4.612518,Seasonal Lightweight Leggings
1,192,4.377689,Athleisure Leggings
2,78,4.341187,Kids Active Shorts
3,210,4.333435,Eco Flared Culottes
4,181,4.330308,Seasonal Lightweight Tunic
5,204,4.259201,Seasonal Flannel Blouse
6,11,4.242475,Double-G Leather Bomber
7,98,4.236441,Kids Beach Cover-up
8,117,4.234217,GG Monogram Silk Scarf
9,99,4.224790,Recycled Swim Shorts


# ***User-User***

In [57]:
# def recommend_items_user_user(user_id: int, user_item_matrix: pd.DataFrame, user_similarity: pd.DataFrame,
#                               products: pd.DataFrame, user_means: pd.Series, n: int = 10, k: int = 20,
#                               similarity_type: str = 'pearson') -> pd.DataFrame:
#     """
#     Generate top-n user-user collaborative filtering recommendations for a user using Pearson correlation.

#     Args:
#         user_id (int): ID of the target user
#         user_item_matrix (pd.DataFrame): User-item matrix
#         user_similarity (pd.DataFrame): User-user similarity matrix (Pearson correlation)
#         products (pd.DataFrame): Products DataFrame
#         user_means (pd.Series): User mean ratings
#         n (int): Number of recommendations to return
#         k (int): Number of similar users to consider
#         similarity_type (str): Type of similarity measure ('pearson')

#     Returns:
#         pd.DataFrame: Recommended products with predicted ratings
#     """
#     # Identify unrated items
#     unrated_items = user_item_matrix.loc[user_id][user_item_matrix.loc[user_id].isna()].index
#     predictions = []
#     similarity_details = []

#     # Ensure common users exist in similarity matrix
#     common_users = user_similarity.index.intersection(user_item_matrix.index)

#     # Predict rating for each unrated item
#     for item in unrated_items:
#         # Find users who rated this item
#         rated_users = user_item_matrix[item].dropna().index
#         rated_users = rated_users[rated_users.isin(common_users)]
#         if len(rated_users) == 0:
#             continue
#         try:
#             # Get top-k similar users who rated this item
#             similar_users = user_similarity[user_id].loc[rated_users].nlargest(k)
#         except KeyError:
#             continue
#         numerator, denominator = 0, 0
#         user_similarities = []
#         for sim_user, sim_score in similar_users.items():
#             if not np.isnan(user_item_matrix.loc[sim_user, item]) and not np.isnan(sim_score):
#                 centered_rating = user_item_matrix.loc[sim_user, item] - user_means.get(sim_user, ratings['stars'].mean())
#                 numerator += sim_score * centered_rating
#                 denominator += abs(sim_score)
#                 user_similarities.append((sim_user, sim_score))
#         if denominator > 0:
#             predicted_rating = user_means.get(user_id, ratings['stars'].mean()) + (numerator / denominator)
#             predicted_rating = min(max(predicted_rating, 0), 5)
#             predictions.append((item, predicted_rating))
#             similarity_details.append((item, user_similarities))

#     # Sort and select top-n recommendations
#     predictions = sorted(predictions, key=lambda x: x[1], reverse=True)[:n]

#     # Fallback to popular products if not enough predictions
#     if len(predictions) < n:
#         popular_products = ratings.groupby('product_id')['stars'].mean().nlargest(n).index
#         popular_products = [pid for pid in popular_products if pid not in user_item_matrix.loc[user_id].dropna().index]
#         popular_predictions = [(pid, ratings[ratings['product_id'] == pid]['stars'].mean()) for pid in popular_products]
#         predictions.extend(popular_predictions[:n - len(predictions)])
#         similarity_details.extend([(pid, []) for pid in popular_products[:n - len(predictions)]])

#     # Merge with product info
#     result = products[products['id'].isin([x[0] for x in predictions])][['id', 'name']].merge(
#         pd.DataFrame(predictions, columns=['id', 'predicted_rating']), on='id'
#     )

#     # Print similarity details
#     print(f"\nChi tiết độ tương đồng ({similarity_type}) cho top-{n} đề xuất của user {user_id}:")
#     for item_id, pred_rating in predictions:
#         item_name = products[products['id'] == item_id]['name'].iloc[0]
#         # Fix the syntax error in the list comprehension
#         sim_scores_list = [sims for item, sims in similarity_details if item == item_id]
#         sim_scores = sim_scores_list[0] if sim_scores_list else []

#         print(f"Sản phẩm {item_id} ({item_name}): Rating dự đoán = {pred_rating:.4f}")
#         if sim_scores:
#             print(f"  Các người dùng tương đồng (top-{k}):")
#             for sim_user, sim_score in sim_scores:
#                 print(f"    - Người dùng {sim_user}: {similarity_type} = {sim_score:.4f}")
#         else:
#             print("  Không có người dùng tương đồng (dùng popular fallback).")

#     return result

In [58]:
# def generate_predictions_for_test_set(test_df, user_item_matrix_train, user_similarity, user_means_train, k=20):
#     predictions = []
#     global_mean = test_df['stars'].mean()
#     for _, row in test_df.iterrows():
#         user_id = int(row['customer_id'])
#         item_id = int(row['product_id'])
#         predicted_rating = np.nan

#         if user_id in user_item_matrix_train.index and item_id in user_item_matrix_train.columns:
#             rated_users = user_item_matrix_train[item_id].dropna().index
#             rated_users = rated_users[rated_users.isin(user_similarity.index)]
#             if len(rated_users) > 0:
#                 try:
#                     similar_users = user_similarity[user_id].loc[rated_users].nlargest(k)
#                     numerator, denominator = 0, 0
#                     for sim_user, sim_score in similar_users.items():
#                         if not np.isnan(user_item_matrix_train.loc[sim_user, item_id]) and not np.isnan(sim_score):
#                             centered_rating = user_item_matrix_train.loc[sim_user, item_id] - user_means_train.get(sim_user, global_mean)
#                             numerator += sim_score * centered_rating
#                             denominator += abs(sim_score)
#                     if denominator > 0:
#                         predicted_rating = user_means_train.get(user_id, global_mean) + (numerator / denominator)
#                         predicted_rating = min(max(predicted_rating, 0), 5)
#                     else:
#                         predicted_rating = user_means_train.get(user_id, global_mean)
#                 except KeyError:
#                     predicted_rating = user_means_train.get(user_id, global_mean)
#             else:
#                 predicted_rating = user_means_train.get(user_id, global_mean)
#         else:
#             predicted_rating = user_means_train.get(user_id, global_mean)
#         predictions.append(predicted_rating)

#     test_df['predicted_rating'] = predictions
#     return test_df

In [59]:
# def evaluate_model(predictions_df):
#     """
#     Đánh giá mô hình bằng cách tính toán RMSE.

#     Args:
#         predictions_df (pd.DataFrame): DataFrame chứa cột 'stars' (thực tế)
#                                       và 'predicted_rating' (dự đoán).

#     Returns:
#         float: Giá trị RMSE.
#     """
#     if predictions_df.empty:
#         print("Không có dự đoán nào để đánh giá.")
#         return float('inf')

#     # Loại bỏ các dòng không thể dự đoán được (predicted_rating là NaN)
#     predictions_df.dropna(subset=['predicted_rating'], inplace=True)

#     if predictions_df.empty:
#         print("Tất cả các dự đoán đều là NaN.")
#         return float('inf')

#     # Tính Mean Squared Error (MSE)
#     mse = mean_squared_error(predictions_df['stars'], predictions_df['predicted_rating'])

#     # Tính Root Mean Squared Error (RMSE)
#     rmse = np.sqrt(mse)

#     return rmse

In [60]:
# products = pd.read_csv("products_info.csv")
# ratings = pd.read_csv("ratings_with_time.csv")

# ratings = preprocess_ratings(ratings)
# cleaned_products, cleaned_ratings = validate_products(products, ratings)

# train_df, test_df = train_test_split(cleaned_ratings, test_size=0.3, random_state=42)
# print(f"Training set: {len(train_df)}")
# print(f"Test set: {len(test_df)}\n")

In [61]:
# # Create user-item matrix for training set
# user_item_matrix_train = create_user_item_matrix(train_df)

# # Mean-center the user-item matrix
# user_item_matrix_train_centered, user_means_train = mean_center_matrix(user_item_matrix_train)

# # Compute user-user similarity (using Pearson correlation on mean-centered matrix)
# print("Huấn luyện mô hình: Tính toán User-User Similarity từ Training Set (Pearson)")
# user_similarity_df = user_item_matrix_train_centered.corr(method='pearson').fillna(0)

# # Save matrices for inspection
# user_item_matrix_train.to_csv("user_item_matrix_train.csv")
# user_item_matrix_train_centered.to_csv("user_item_matrix_train_centered.csv")
# user_means_train.to_csv("user_means_train.csv")
# user_similarity_df.to_csv("user_similarity.csv")

In [62]:
# # Example: Generate recommendations for user 1
# recommendations = recommend_items_user_user(
#     user_id=1,
#     user_item_matrix=user_item_matrix_train,
#     user_similarity=user_similarity_df,
#     products=cleaned_products,
#     user_means=user_means_train,
#     n=10,
#     k=10,
#     similarity_type='pearson'
# )

# print(f"\nTop-10 recommendations for user 1:\n{recommendations}")

In [63]:
# # Generate predictions for test set
# print("Tạo dự đoán cho Test Set")
# predictions_df = generate_predictions_for_test_set(
#     test_df.copy(),
#     user_item_matrix_train,
#     user_similarity_df,
#     user_means_train,
#     k=25
# )

# print("Example:")
# print(predictions_df.dropna(subset=['predicted_rating']).head(20))

In [64]:
# print("\n--- Evaluate model ---")
# rmse_score = evaluate_model(predictions_df)
# print(f"Root Mean Square Error (RMSE): {rmse_score:.4f}")

In [65]:
# # Tính Precision@K và Recall@K
# k_value = 10
# relevance_threshold = 4.0

# # Gọi hàm đánh giá xếp hạng
# precision, recall = calculate_ranking_metrics(
#     predictions_df,
#     k=k_value,
#     threshold=relevance_threshold
# )

# print(f"\n--- All metrics rating (with K={k_value} và and revevant threshold >= {relevance_threshold}) ---")
# print(f"Precision at Top-{k_value} (Precision@{k_value}): {precision:.2%}")
# print(f"\nRecall at Top-{k_value} (Recall@{k_value}): {recall:.2%}")